# 安装依赖

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
!pip install pysrt
from tqdm import tqdm
import pysrt
google_drive=False
import os

# 选择翻译方式

In [ ]:
#@title 翻译模型翻译方式，同时加载模型，查看可用语言（最后一步的源语言和目标语言只能从可用语言里面选，每个模型可用语言各不相同）
!pip install dl_translate
import dl_translate as dlt

翻译模型 = "facebook/nllb-200-distilled-600M" # @param ["facebook/m2m100_418M", "facebook/mbart-large-50-many-to-many-mmt", "facebook/nllb-200-distilled-600M"]

mt = dlt.TranslationModel(翻译模型,device="auto")
print("可用语言：")
print("\n".join(mt.available_languages()))

In [ ]:
#@title 谷歌翻译方式（免费调用，不稳定，最后一步的源语言和目标语言只能从可用语言里面选）
!pip install googletrans==3.1.0a0
import googletrans
translator = googletrans.Translator()
print("可用语言：")
print("\n".join(googletrans.LANGUAGES))

In [ ]:
#@title ChatGLM3翻译方式（系统提示词中"{}"将被替换为目标语言）
系统提示词 = "\u4F60\u662F\u4E00\u4E2A\u4E13\u4E1A\u7684\u7FFB\u8BD1\uFF0C\u4F60\u9700\u8981\u4FDD\u7559unity\u4E13\u7528\u672F\u8BED\u4E0D\u7FFB\u8BD1\uFF0C\u5176\u5B83\u6240\u6709\u8BCD\u90FD\u5FC5\u987B\u7FFB\u8BD1\u3002\u8BF7\u9010\u884C\u7FFB\u8BD1\u4E0B\u9762\u7684\u6587\u672C\u5230{0}\uFF0C\u6CE8\u610F\u4FDD\u7559\u6570\u5B57\u548C\u6362\u884C\u7B26\uFF0C\u8BF7\u52FF\u81EA\u884C\u521B\u5EFA\u5185\u5BB9\uFF0C\u9664\u4E86\u7FFB\u8BD1\uFF0C\u4E0D\u8981\u8F93\u51FA\u4EFB\u4F55\u5176\u4ED6\u6587\u672C\u3002" #@param {type: "string"}
!pip install transformers sentencepiece
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True, device='cuda')
model = model.eval()


# 连接到谷歌云盘

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
google_drive=True
谷歌云盘保存到 = '/content/drive/MyDrive/ChatGLM3\u7FFB\u8BD1.srt'  #@param {type: "string"}

# 翻译

In [ ]:
源文件 = '\u539F\u89C6\u9891.srt'  #@param {type: "string"}
保存到 = '翻译.srt'  #@param {type: "string"}
源语言 = 'en'  #@param {type: "string"}
目标语言 = 'zh-cn'  #@param {type: "string"}
翻译方式 = "\u8C37\u6B4C\u7FFB\u8BD1" #@param ["谷歌翻译", "大模型翻译","ChatGLM3翻译","不翻译"]
恢复文件 = '\u6062\u590D\u6587\u4EF6.srt'  #@param {type: "string"}
恢复间隔 = 20  #@param {type: "integer"}

def translate(sentence):
    if 翻译方式=="大模型翻译":
        return mt.translate(sentence, source=源语言, target=目标语言)
    elif 翻译方式=="谷歌翻译":
        return translator.translate(sentence, src=源语言, dest=目标语言).text
    elif 翻译方式=="ChatGLM3翻译":
        response,now_history=model.chat(tokenizer,sentence,top_p=1,temperature=0.01,history=[{"role": "system", "content": 系统提示词.format(目标语言)}])
        return response
    elif 翻译方式=="不翻译":
        return sentence
def merge_and_translate_with_progress(subtitles):
    current_sentence = ""
    current_start = None
    current_end = None

    # 是否进行恢复
    if 恢复文件:
        if not os.path.exists(恢复文件):
            # Create the file if it does not exist
            with open(恢复文件, "w") as f:pass
        # 读取恢复文件
        recovered_subs = pysrt.open(恢复文件, encoding='utf-8')
        recover_index = 0
        if len(recovered_subs)!=0:
            # 找到恢复位置，跳过已翻译部分
            for i, sub in enumerate(subtitles):
                if sub.end == recovered_subs[-1].end:
                    recover_index = i + 1
                    break
        translated_subs = recovered_subs
    else:
        recover_index = 0
        translated_subs = pysrt.SubRipFile()

    # 使用 tqdm 显示进度条
    progress_bar = tqdm(total=len(subtitles)-recover_index, desc="翻译中", unit="个", mininterval=5)

    for i in range(recover_index, len(subtitles)):
        now_sub=subtitles[i]
        if i<len(subtitles)-1:
            next_sub=subtitles[i+1]
        #分割条件：当前句子结尾有“.”，下一个句子start与当前句子end不相等，已经是最后一个句子
        #若符合条件，则对当前句子进行分割
        if current_start is None:
            current_start=now_sub.start
        current_end=now_sub.end
        current_sentence+=now_sub.text+" "
        if i==len(subtitles)-1 or now_sub.text[-1]=="." or next_sub.start!=now_sub.end:
            translated_text = translate(current_sentence.strip())
            # 创建新的字幕项并添加到结果
            translated_sub = pysrt.SubRipItem(index=len(translated_subs) + 1,
                                             start=current_start, end=current_end,
                                             text=translated_text)
            translated_subs.append(translated_sub)

            # 重置参数
            current_sentence = ""
            current_start = None
            current_end = None

            if 恢复文件 and (i + 1) % 恢复间隔 == 0:
                translated_subs.save(恢复文件, encoding='utf-8')
        # 更新进度条
        progress_bar.update(1)

    # 关闭进度条
    progress_bar.close()

    return translated_subs

# 读取SRT文件
subs = pysrt.open(源文件, encoding='utf-8')

# 获取翻译后的字幕
translated_subs = merge_and_translate_with_progress(subs)

# 保存翻译后的SRT文件
translated_subs.save(保存到, encoding='utf-8')
if google_drive:
  translated_subs.save(谷歌云盘保存到, encoding='utf-8')